## Install Lib API

In [1]:
! pip install api-0.1.0-py3-none-any.whl

Processing ./api-0.1.0-py3-none-any.whl
     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 48.8 MB/s 
  Created wheel for kneebow: filename=kneebow-0.1.1-py3-none-any.whl size=3592 sha256=eb6e170612f0bec1370370f4c25092dfa3f3f1dac6358d3d1e62737c6aa56331
  Stored in directory: /root/.cache/pip/wheels/df/c8/26/01654920ba2a566c78a34582bb013ba47e0eb8ce358008954c
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=7886fa80f5087e57c664ff5283eeb054e126bd626c16457147ea700eb35f193c
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built kneebow pyspark


In [2]:
! python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2 MB 16.8 MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-py3-none-any.whl size=21186282 sha256=0fe4a64dc02f572923edfb79b56a687923717973e0f153593826bb186b1df304
  Stored in directory: /tmp/pip-ephem-wheel-cache-01th6sh3/wheels/c3/f9/0c/5c014a36941a00f5df5fc0756cb961d7c457a978e697a6ce3b
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


## Import libs

In [3]:
import pandas as pd
import spacy
import nltk
nltk.download('stopwords')
from api import nlextract

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


## Read Excel file

In [4]:
df = pd.read_excel("/content/Conversas Sodexo.xlsx", engine='openpyxl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179685 entries, 0 to 179684
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Codigo                 179685 non-null  int64         
 1   CHAT_Protocolo         179685 non-null  int64         
 2   TEM                    179685 non-null  int64         
 3   ChatID                 179685 non-null  int64         
 4   ExpertID               179685 non-null  int64         
 5   ExpertFullName         179685 non-null  object        
 6   CategoryName           179685 non-null  object        
 7   ClosedBy               179614 non-null  object        
 8   DateAnswered           179685 non-null  float64       
 9   DateFinished           179541 non-null  float64       
 10  CustomerID             179685 non-null  int64         
 11  CustomerName           179644 non-null  object        
 12  CustomerEmail          179685 non-null  obje

#### See the first line of dataset

In [42]:
df.head(1)

,Codigo,CHAT_Protocolo,TEM,ChatID,ExpertID,ExpertFullName,CategoryName,ClosedBy,DateAnswered,DateFinished,CustomerID,CustomerName,CustomerEmail,DataInsercao,Status,Resolution,CustomerFieldA,CustomerFieldC,CustomerFieldD,CustomerFieldE,CustomerFieldG,CustomerFieldH,CustomerFieldU,Link,UniqueID,Type,MsgTipo,ClassificacaoTags,ClassificacaoTreeName,CustomerName_1,ExpertFullName_2,Date,MÊS,Message,Column1,alto_atrito,cancelamento
0,7,350229426,350229426,31646020,202084,DAINA ANDREZA DE SOUZA,Chat - Atendimento Robô,Encerrado pelo consumidor,44470.349259,44470.354734,80596540,AMANDA,administrativo@grupoapec.com.br,44559.410706,2,2.0,7075888000170,NaN,NaN,NaN,NaN,NaN,NaN,sodexoa,3.164602e+16,1201,1201,NaN,NaN,AMANDA,NaN,2021-10-01 08:22:56,OUTUBRO,bloqueio de cartão,NaN,[],[]


### Call the functions inside of wheels file

In [6]:
extractor = nlextract.NLExtractor()

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Lista de Palavras Procuradas

#### Formato Exemplo:
  variavel = {coluna_nome : [lista de palavras procuradas],
              coluna_nome : [lista de palavras procuradas]}

In [36]:
mydict = {
    'alto_atrito': ['ainda não', 'não chegou', 'até agora', 'até o momento', 'atraso', 'reclamação', 'pelo amor de Deus', 'procon', \
                   'péssimo', 'cansado', 'incompetente', 'ouvidoria', 'frustração', 'absurdo', 'porra', 'PQP', 'poxa vida', 'horrível',\
                   'ridículo', 'decepção', 'humilhação', 'FDP', 'merda', 'triste', 'bosta', 'protocolo'],
    'cancelamento': ['cancelar minha viagem', 'quero cancelar', 'cancelamento', 'desejo cancelar', 'quero desativar', 'verificado']
          }

#### Coluna titulo do texto

In [37]:
coluna_texto = 'Message'

#### Colocar em caixa baixa (lower case) a coluna de texto

In [38]:
df[coluna_texto] = df[coluna_texto].str.lower()

#### Aplicar função de Caça Palavras no Texto

In [39]:
try:
  for key in mydict:
    df[key] =  df[coluna_texto].apply(lambda x: extractor.pattern_matcher(x,mydict[key],mode="dictionary", anonymizer=False,custom_anonymizer=f'<{key}>'))
except IOError as e:
  print(f'não tem mais listas para rodar dados {e}')
  pass

#### Avaliação das colunas do Caça Palavras geradas

In [40]:
df[['Message','alto_atrito','cancelamento']].head(50)

,Message,alto_atrito,cancelamento
0,bloqueio de cartão,[],[]
1,"olá, seja bem-vindo (a) ao chat da sodexo. por favor anote o número do seu protocolo 350229426, para iniciarmos... ah, e não esqueça de avaliar ao final do atendimento! :wink:","[{'entity': 'protocolo', 'value': 'protocolo', 'type': 'protocolo', 'startIndex': 75, 'endIndex': 85}]",[]
2,bom dia,[],[]
3,gostaria de solicitar o desbloqueio de dois cartões,[],[]
4,confirmação de dados para pedido,[],[]
5,"olá, seja bem-vindo (a) ao chat da sodexo. por favor anote o número do seu protocolo 350229610, para iniciarmos... ah, e não esqueça de avaliar ao final do atendimento! :wink:","[{'entity': 'protocolo', 'value': 'protocolo', 'type': 'protocolo', 'startIndex': 75, 'endIndex': 85}]",[]
6,bom dia,[],[]
7,preciso saber sobre um pedido que foi feito á mais de uma semana,[],[]
8,nº - 26281401/21,[],[]
9,"bom dia, amanda tudo bem?",[],[]


## Salvar o Resultado em um CSV

In [41]:
filename = 'sodexo_tratado'
file_save = f'{filename}.csv'
df.to_csv(file_save, sep=';',encoding='utf-8',index=False)